# Single Species Hatchery Continuation
This is a continuation of the notebook [single_secies.ipynb](single_species.ipynb) (split into multiple notebooks for performance reasons).

In [ ]:
source("hatchery.R") # imports mizer
library(mizerExperimental)
library(ggplot2)
library(plotly)

## Predation Model
We return to our cannibalistic single species model and investigate the impact of a smaller hatchery on the behaviour of the model:

In [ ]:
params <- newSingleSpeciesParams()
params <- setInteraction(params, 1)
species_params(params)$interaction_resource <- 0.8
params <- steady(params)

species_params(params)

plotlySpectra(params)

In [ ]:
# determine more reasonable hatchery input by fraction of total number of mature
# individuals maturity
N_mat <- getN(params, min_w=species_params(params)$w_mat)
View(N_mat)

# number of individuals to add from hatchery as a % of N_mat, i.e. ratio=1 => 
# add one individual for every mature individual in population
ratio <- 1
#! what is a sensible ratio? is this even a sensible way to scale?

# define hatchery_params
hatchery_params = data.frame(species="Target species", mu=4, sigma=0.5, annual_N=N_mat * ratio)
View(hatchery_params) # display hatchery params

# add one year of this input
sim <- projectHatchery(params, t_max=1, t_save=1, dt=1)
plotlySpectra(sim) # plot biomass spectra after hatchery input

In [ ]:
sim <- project(sim, t_max=6, t_save=0.05, dt=0.01)
plotlyBiomassRelative(sim)

In [ ]:
# get final biomass/initial biomass from sim
getChange(sim)

In [ ]:
# # save a series of plots to generate an animation ----

# # get vector of times from projection
# times <- getTimes(sim)
# # remove first time from animation (steady state before hatchery input)
# times <- times[2:length(times)]

# for (t in times) {
#     print(sprintf("plots/spectra2/%.2f.png", t)) # debug

#     # create plot of spectra at time `t`
#     p <- plotSpectra(sim, time_range=t) + ggtitle(sprintf("Time = %.2f", t))

#     # save the plots
#     ggsave(sprintf("plots/spectra2/%.2f.png", t),
#         plot = p,
#         device="png",
#         dpi=160, # retina dpi = 320
#         height=5,
#         width=8
#         # scale=1
#         )
# }

# #! to compile gif run the following in bash (with ImageMagick installed) 
# #! `convert -resize 30% -delay 15 -loop 0 *.png spectra2.gif`

Animation:

![spectra anim](plots/spectra2.gif)

Projection 

In [ ]:
sim2 <- projectHatchery(params, t_max=20, t_save=0.5, dt=0.1)
plotlyBiomassRelative(sim2)

In [ ]:
getChange(sim2)

In [ ]:
# plot spectra at end of simulation (approaching steady state)
plotlySpectra(sim2)

## Seasonal Hatchery
Before now, we have been considering a simple model where the hatchery is assumed to be operating year round at a constant rate. In reality however, species often mate in specific seasons. As such, here we introduce a model where the hatchery operates as a sin through the year

In [ ]:
# define new hatchery time dependence function (default is constant 1)
hatchery_T <- function(t) {
    return(1 - sign(sin(2 * pi * t))) # square wave
}

# alternatively, for square waves which are on during select times in the year
# hatchery_T <- function(t) {
#     # number between 0 and 1 (start of hatchery release each year - end of year)
#     threshold <- 0.75

#     if (t %% 1 > threshold) {
#         return(1/threshold)
#     }
#     return(0)
# }

In [ ]:
sim <- projectHatchery(params, t_max=20, t_save=0.1, dt=0.01)
plotlyBiomassRelative(sim)

In [ ]:
getChange(sim)

In [ ]:
# # save a series of plots to generate an animation ----

# # get vector of times from projection
# times <- getTimes(sim)
# # remove first time from animation (steady state before hatchery input)
# times <- times[2:length(times)]

# for (t in times) {
#     print(sprintf("plots/spectra3/%.2f.png", t)) # debug

#     # create plot of spectra at time `t`
#     p <- plotSpectra(sim, time_range=t) + ggtitle(sprintf("Time = %.2f", t))

#     # save the plots
#     ggsave(sprintf("plots/spectra3/%.2f.png", t),
#         plot = p,
#         device="png",
#         dpi=100, # retina dpi = 320
#         height=5,
#         width=8
#         # scale=1
#         )
# }

# #! to compile gif run the following in bash (with ImageMagick installed) 
# #! `convert -delay 10 -loop 0 *.png spectra3.gif`

Animation:

![spectra anim](plots/spectra3.gif)

## Fishing the model
Now, we have seen that the addition of a hatchery can significantly increase the biomass in the system, it remains to be seen however, how much this affects fishing yield.

In [ ]:
# define params object with initial fishing effort
params2 <- params
initial_effort(params2) <- 1
params2 <- steady(params2)
plotlySpectra(params2)

In [ ]:
# demonstrate that steady state biomass of the fished model differs from the
# steady state biomass of the old (unfished) model by comparing directly
getBiomass(params)
getBiomass(params2)

In [ ]:
#! TODO: compare fishing yield on non-hatchery model to fishing yield on hatchery model
baseline <- project(params2, t_max=15, t_save=0.2, dt=0.05)
hatchery <- projectHatchery(params2, t_max=15, t_save=0.2, dt=0.05)

In [ ]:
plotlyBiomassRelative(baseline)

In [ ]:
plotBiomassRelative(hatchery)
ggsave(sprintf("single species/fish-relbio-season.png",t), dpi=300, width = 6, height = 4)

In [ ]:
plotYieldRelative(hatchery, baseline)
ggsave(sprintf("single species/fish-relyield-season.png",t), dpi=300, width = 6, height = 4)



In [ ]:
# plotlyPredMort()

In [ ]:
plotlySpectra(hatchery)